# Eléments de programmation

Pour mener à bien un calcul algoritmique le nombre d'éléments de langage n'est pas très important et peut se résumer aux 3 syntaxes suivantes

* Le choix conditionnel **if**-**elseif**-**else**-**end**.
* La boucle **for**-**end**.
* La boucle **while**-**end**


# if ... elseif ... else ... end

Un choix simple si le test est vrai (k==1) alors le bloc d'instruction est évalué

In [1]:
k=1;
if k==1
    println("k=1")
end

k=1


Le **else** permet de donner un résultat par défaut...

In [2]:
k=1;
if k!=1
    println("k<>1")
else
    println("k=1")
end

k=1


Une succession de **elseif** permet de choisir parmi plusieurs critères, dans la succession des bloks de **if** et **elseif** le premier qui est "vrai" est évaluer et l'instruction s'arrète.

In [3]:
k=2;
if k==1
    println("k=1")
elseif k>1
    println("k>1")
else 
    println("k<1")
end

k>1


# La boucle for

Elle peut se définir à l'aide d'itérateurs ou de tableaux de valeurs les syntaxes "=" ou "in" sont équivallentes

In [4]:
for i=1:10
    println(i)
end

1
2
3
4
5
6
7
8
9
10


In [5]:
for i in ["un" 2 im]
    println(typeof(i))
end

ASCIIString
Int64
Complex{Bool}


La commande **breack** permet de sortir d'une boucle à tout moment

In [6]:
for i = 1:1000
    println(i)
    if i >= 5
       break
    end
end

1
2
3
4
5


La commande **continue** permet elle de courtcircuiter la boucle en court et de passer à la valeur suivante

In [7]:
for i = 1:10
    if i % 3 != 0 # i modulo 3 different de 0
        continue
    end
    println(i)
end

3
6
9


# La boucle while

Tant que le test est "vrai" le blok est évalué, le test se faisant en entrée de blok

In [8]:
k=0;
while k<10
    k+=1
    println(k)
end

1
2
3
4
5
6
7
8
9
10


De même que la boucle **for** les commandes **breack** et **continue** sont valable...

In [ ]:
\par